In [1]:
# Dependencies
import requests
import json
import os
import pandas as pd
import numpy as np


# Google developer API key
gkey = os.getenv("GKEY")


In [2]:
# import data from countries csv
countries = pd.read_csv("csv/countries.csv")

In [3]:
countries.head()

,Unnamed: 0,country,country_id
0,0,Afghanistan,1
1,1,Albania,2
2,2,Algeria,3
3,3,Angola,4
4,4,Argentina,5


In [5]:
countries["Lat"] = ""
countries["Lng"] = ""
countries.head()

,Unnamed: 0,country,country_id,Lat,Lng
0,0,Afghanistan,1,,
1,1,Albania,2,,
2,2,Algeria,3,,
3,3,Angola,4,,
4,4,Argentina,5,,


In [6]:
def google_api_request(url, params=None):
    params = params or {}
    params['key'] = gkey
    result = requests.get(url, params=params).json()
    if 'error_message' in result:
        raise Exception(result['error_message'])
    return result['results']

def geocode_results(address):
    return google_api_request('https://maps.googleapis.com/maps/api/geocode/json', params={
        'address': address,
    })

def place_results(lat, lng, radius=8000, place_type=None, keyword=None, rankby=None):
    params = {
        'location': f"{lat},{lng}",
        'radius': radius,
    }
    if place_type:
        params['type'] = place_type
    if keyword:
        params['keyword'] = keyword
    if rankby:
        params['rankby'] = rankby
        del params['radius']
    return google_api_request('https://maps.googleapis.com/maps/api/place/nearbysearch/json', params)

In [8]:
for index, row in countries.iterrows():
    city = row['country']
    results = geocode_results(f"{city}")

    countries.loc[index, "Lat"] = results[0]["geometry"]["location"]["lat"]
    countries.loc[index, "Lng"] = results[0]["geometry"]["location"]["lng"]


In [9]:
countries.head()

,Unnamed: 0,country,country_id,Lat,Lng
0,0,Afghanistan,1,33.9391,67.71
1,1,Albania,2,41.1533,20.1683
2,2,Algeria,3,28.0339,1.65963
3,3,Angola,4,-11.2027,17.8739
4,4,Argentina,5,-38.4161,-63.6167


In [10]:
countries.columns

Index(['Unnamed: 0', 'country', 'country_id', 'Lat', 'Lng'], dtype='object')

In [11]:
countries.drop(columns = ['Unnamed: 0'],axis=1)

,country,country_id,Lat,Lng
0,Afghanistan,1,33.9391,67.71
1,Albania,2,41.1533,20.1683
2,Algeria,3,28.0339,1.65963
3,Angola,4,-11.2027,17.8739
4,Argentina,5,-38.4161,-63.6167
5,Armenia,6,40.0691,45.0382
6,Australia,7,-25.2744,133.775
7,Austria,8,47.5162,14.5501
8,Azerbaijan,9,40.1431,47.5769
9,Bahrain,10,26.0667,50.5577
